In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency


In [ ]:
df = pd.read_csv('Asurion_clean_data.csv')
iphone_df = (df[df['phone model'].str.startswith('apple')])
iphone_df_new = iphone_df[iphone_df['phone model'].str.endswith('13', '14')]

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder


# Step 1: Data preparation
data = iphone_df_new 
encoder = OneHotEncoder()
model_encoded = encoder.fit_transform(data[['phone model']])

# Combine the encoded feature with the remaining numerical features
X = pd.concat([data.drop('phone model', axis=1), pd.DataFrame(model_encoded.toarray())], axis=1)
y = data['claim'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

# Step 2: Feature engineering
#scaler = MinMaxScaler()
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.transform(X_test)

# Step 3: Model building
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size=32)

# Step 4: Model evaluation
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)